In [ ]:
from dopapy.core.services.proxy.iservice import IServiceProxy
from numpy import ndarray
from dopapy import *
from dopapy.trading import *
import dopapy.types as dd
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import warnings
warnings.filterwarnings('ignore') #Thats just to ignore warnings that are irrelevant for this type of sample

   
n = 100
init_logging()
mnist = input_data.read_data_sets("/data/mnist/", one_hot=True)


retina_service_desc = dd.ServiceDescriptor(input_descriptors=[dd.TensorDescriptor(size=[-1,28,28,1], store_file_prefix="consumer1")],
                                             service_role=dd.ServiceRole.CONSUMER,
                                             service_type=dd.ServiceType.Learning.GRADIENT,
                                             remote_sources=[get_counter_quote('https://127.0.0.1:8500', 0)])

v1_service_desc = dd.ServiceDescriptor(input_descriptors=[retina_service_desc],
                                             service_role=dd.ServiceRole.CONSUMER,
                                             service_type=dd.ServiceType.Learning.GRADIENT,
                                             remote_sources=[get_counter_quote('https://127.0.0.1:8501', 0)])

classifier_service_desc = dd.ServiceDescriptor(input_descriptors=[v1_service_desc],
                                             output_descriptors=[dd.TensorDescriptor(size=[-1,10])],
                                             service_role=dd.ServiceRole.CONSUMER,
                                             service_type= dd.ServiceType.Learning.LABELED,
                                             remote_sources=[get_counter_quote( "https://127.0.0.1:8502", 5)])

consumer_session = create_session('password123456', '/data/dopamine/keystore_dir_a/', '127.0.0.1',
                                  int(100E18), int(10E18), web_server_port=8503)

consumer_reward_desc = create_reward_descriptor(min_gas_price=int(400E9),max_dopamine_price=1000)
consumer_reward_desc.add_payment(dd.RewardPaymentType.FORWARD, min_price=int(0), in_advance_quantity=1)
consumer_reward_desc.add_payment(dd.RewardPaymentType.RETURN_GRADIENT, min_price=20,in_advance_quantity=n)
consumer_reward_desc.add_payment(dd.RewardPaymentType.UPDATE, min_price=30,in_advance_quantity=n)

consumer_services = create_quote_services(
    session=consumer_session,
    qty=1,
    side=BUY,
    service_descriptor=classifier_service_desc,
    reward_descriptor=consumer_reward_desc)
service = consumer_services.create_service()
batch_size = 500


def print_accuracy(multi_service: IServiceProxy, x: ndarray, y: ndarray, step):
    predict = np.argmax(multi_service.call(dd.Tensor(x)).n_array, 1)
    target = np.argmax(y, 1)
    accuracy = np.sum(predict == target)/len(target)
    print( "step ({0}) - *******************************************  accuracy : {1}".format(step,accuracy))


for step in range(n):
    batch_data, batch_label = mnist.train.next_batch(batch_size)
    batch_data = batch_data.reshape(batch_size, 28, 28, 1)
    print_accuracy(service, batch_data, batch_label, step)
    service.train([batch_data], [batch_label], True, True)



Extracting /data/mnist/train-images-idx3-ubyte.gz
Extracting /data/mnist/train-labels-idx1-ubyte.gz
Extracting /data/mnist/t10k-images-idx3-ubyte.gz
Extracting /data/mnist/t10k-labels-idx1-ubyte.gz
step (0) - *******************************************  accuracy : 0.078
